# Loop 68 Analysis: Submission Failure Investigation

**Problem**: exp_064 (revert_exp030) failed with "Evaluation metric raised an unexpected error"

**Key Questions**:
1. Why did the submission fail when exp_030 worked?
2. What is the CV-LB relationship?
3. What strategies can break the 0.087 floor?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Submission history with CV and LB scores
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
    {'exp': 'exp_035', 'cv': 0.0098, 'lb': 0.0970},
]

df = pd.DataFrame(submissions)
print('Submission History:')
print(df.to_string(index=False))
print(f'\nBest LB: {df["lb"].min():.4f} (exp_030)')
print(f'Best CV: {df["cv"].min():.4f} (exp_030)')
print(f'Target LB: 0.0347')

In [ ]:
# CV-LB Relationship Analysis
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])

print('=== CV-LB Relationship ===')
print(f'Linear fit: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R-squared = {r_value**2:.4f}')
print(f'Intercept = {intercept:.4f}')
print(f'Target LB = 0.0347')
print(f'\nInterpretation:')
print(f'  - Even at CV=0, expected LB = {intercept:.4f}')
print(f'  - Intercept ({intercept:.4f}) > Target (0.0347): IMPOSSIBLE to reach target by CV optimization alone!')
print(f'  - Required CV to hit target: (0.0347 - {intercept:.4f}) / {slope:.2f} = {(0.0347 - intercept) / slope:.4f}')
print(f'\nThe intercept represents STRUCTURAL distribution shift that no model tuning can fix.')

In [ ]:
# Plot CV vs LB
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], s=100, alpha=0.7)

# Add regression line
cv_range = np.linspace(0, df['cv'].max() * 1.1, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f} (R2={r_value**2:.3f})')

# Add target line
plt.axhline(y=0.0347, color='g', linestyle=':', linewidth=2, label='Target LB = 0.0347')

# Add intercept line
plt.axhline(y=intercept, color='orange', linestyle='--', alpha=0.5, label=f'Intercept = {intercept:.4f}')

plt.xlabel('CV Score (MSE)')
plt.ylabel('LB Score (MSE)')
plt.title('CV vs LB Relationship - All Submissions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print('\nKey insight: The intercept is HIGHER than the target (0.0347).')
print('This means even with perfect CV=0, we would still miss the target.')

In [ ]:
# Gap Analysis
print('=== Gap Analysis ===')
print(f'Best LB achieved: 0.0877 (exp_030)')
print(f'Target LB: 0.0347')
print(f'Gap: {0.0877 - 0.0347:.4f} ({(0.0877 - 0.0347) / 0.0347 * 100:.1f}% above target)')
print(f'\nIntercept: {intercept:.4f}')
print(f'Gap from intercept to target: {intercept - 0.0347:.4f}')
print(f'\nTo reach target, we need to REDUCE THE INTERCEPT, not just improve CV.')
print(f'\nStrategies to reduce intercept:')
print('1. Extrapolation detection - blend toward mean when far from training distribution')
print('2. Uncertainty-weighted predictions - conservative when uncertain')
print('3. Physics-informed constraints that generalize to unseen solvents')
print('4. Domain adaptation techniques')
print('5. Study what top public kernels do differently')

In [ ]:
# Check submission file format
submission = pd.read_csv('/home/submission/submission.csv')
print('=== Submission File Analysis ===')
print(f'Shape: {submission.shape}')
print(f'Columns: {submission.columns.tolist()}')
print(f'\nData types:')
print(submission.dtypes)
print(f'\nAny NaN values: {submission.isna().sum().sum()}')
print(f'Any inf values: {np.isinf(submission[["target_1", "target_2", "target_3"]]).sum().sum()}')
print(f'\nValue ranges:')
for col in ['target_1', 'target_2', 'target_3']:
    print(f'  {col}: min={submission[col].min():.6f}, max={submission[col].max():.6f}')
print(f'\nTask distribution: {submission["task"].value_counts().to_dict()}')
print(f'Fold distribution: {sorted(submission["fold"].unique())}')
print(f'\nFirst 5 rows:')
print(submission.head())

In [ ]:
# Check if predictions sum to ~1 (they should for yield fractions)
submission['sum'] = submission['target_1'] + submission['target_2'] + submission['target_3']
print('=== Prediction Sum Analysis ===')
print(f'Sum statistics:')
print(f'  Min: {submission["sum"].min():.4f}')
print(f'  Max: {submission["sum"].max():.4f}')
print(f'  Mean: {submission["sum"].mean():.4f}')
print(f'  Std: {submission["sum"].std():.4f}')
print(f'\nRows where sum < 0.5: {(submission["sum"] < 0.5).sum()}')
print(f'Rows where sum > 1.5: {(submission["sum"] > 1.5).sum()}')
print(f'\nNote: Predictions are Product 2, Product 3, SM (starting material).')
print('They should roughly sum to 1 (mass balance), but not exactly due to side products.')